# **人工智能实验报告 第五周**


## **一、实验题目**
编写程序，分别使用$A*$算法和$IDA*$算法实现15-puzzle问题：通过滑动空格使得4*4的矩阵从初始状态转换为目标状态。

## **二、实验内容**

本次实验分别采用了$A*$算法和$IDA*$算法实现，同时启发函数采用了曼哈顿和线性冲突的计算方式，进行剪枝减少搜索的空间与时间。尤其对于第5，6的`start_state`，需要采用线性冲突的计算方式才可以有效的计算出结果。

### **1启发函数设计**

启发函数用于评估当前状态距离目标状态的估计成本，为公式$f(n)=g(n)+h(n)$中的$h(n)$用于指导启发式的搜索，进行剪枝减少不必要的搜索。

#### **1.1曼哈顿距离**

- **原理**：对于每个非空白的拼图，计算当前坐标与目标坐标之间的曼哈顿距离，具体公式：

  $d = |x_{goal} - x_{current}| + |y_{goal} - y_{current}|$ \
  最后将得到的结果累加起来得到需要的$h(n)$即
  $h(n)=\sum d$
- **评价**：曼哈顿距离计算了每个块的直观移动距离，即每个块在水平和垂直方向上移动到目标位置需要的步数，但是忽略了块被阻挡的情况，故曼哈顿距离计算启发函数本质上低估了实际需要的步数
- **实现**：
  - 函数`get_goal_position`返回`goal_state`的坐标，用于曼哈顿计算
  - 函数`manhattan_distance`遍历所有非空滑块，计算曼哈顿距离并累加返回


#### **1.2线性冲突**

- **原理**：线性冲突是在曼哈顿距离的基础上的补充，专门针对同一行或同一列中存在互相阻挡的情况进行计算。若两个处于同一行或同一列的滑块，均应移动到改行或列且他们当前的顺序与目标顺序相反，则这两个滑块在达到目标时必定至少有一个需要绕行，从而增加额外的移动步数，具体计算为：$$h(n)=h_{manhattan}+2\times h_{conflict}$$
- **评价**：线性冲突优化了曼哈顿距离没有考虑阻挡的情况，更加精确估计需要的步数，但是线性冲突仍然低估了真是移动代价
- **实现**：
  - 函数`linear_conflict`：计算线每个滑块的性冲突数
  - 函数`heuristic`：冲突数$\times2$后与曼哈顿距离相加返回作为最终的$h(n)$

### **2搜索算法设计**

搜索算法是根据启发函数以及公式$f(n)=g(n)+h(n)$进行剪枝后搜索，从而找到符合目标状态的解。

#### **2.1A*算法**

- **原理**：$A*$算法使用最佳优先搜索，核心使用了评价函数：$$f(n)=g(n)+h(n)$$
  其中：
  - $g(n)$：从初始状态到当前状态的实际代价，即移动步数
  - $h(n)$：从当前状态到目标状态的启发估计代价
- **实现**：
  - **节点**：每个`state`使用类`PuzzleNode`封装，包含当前状态`state`，父节点`parent`，移动操作`move`，深度`depth`，以及总代价`cost`，同时包含一个比价函数用于返回最小堆顶的`state`
  - **open_set与closed_set**：
    1. `open_set`使用`heaqp`模块实现优先队列，保证每次扩展是当前`cost`最小的状态
    2. `closed_set`用于记录访问过的`state`，防止出现重复搜索
  - **邻居生成**：函数`get_neighbor`根据空白块的位置以及其可以移动的所有情况作为相邻状态，生成`neighbor`
  - **搜索**：
    1. 从初始状态开始，计算启发值，并且构造为根节点进入`open_set`
    2. 不断循环从`open_set`中取出`f`最小的节点并且检查是否达到`goal_state`
    3. 若未达到则扩展该节点的所有邻居节点，计算新的`depth`($g(n)$)，以及`cost`($f(n)$)，并且将节点加入堆中，更新`closed_set`
    4. 若达到目标节点，回溯查找`move`过程

#### **2.2IDA*算法**

- **原理**：$IDA*$算法结合了深度优先搜索与$A*$算法的优点，通过逐步加深搜索界限的方式，避免了$A*$算法中大量状态的内存开销。
  
  - 每轮搜索设置一个界限`bound`，表示当前允许的最大代价 $f(n)$
  - 每次从初始状态开始深度优先搜索，当遇到某节点的 $f(n)$ 超过`bound`时，返回该分支中最小的超界限值作为下一轮搜索的新的`bound`。

- **实现**：
  - **初始界限设置**：
    - 初始的`bound`取为初始状态的启发函数值
  
  - **递归搜索函数**：
    - 使用函数：`ida_star_search`实现深度优先迭代搜索
    - 在每个递归节点中：
      1. 计算 $f = g + h$，若 $f > bound$，则返回该值用于更新下一轮的界限
      2. 若当前状态为目标状态，则返回成功并输出解路径
      3. 否则，调用 `get_neighbor` 遍历所有邻居状态
         - 若邻居已在路径中则跳过
         - 否则递归搜索该邻居，并持续记录最小超界限值
  
  - **迭代界限更新**：
    - 主函数：`ida_star_puzzle`的流程如下：
      1. 初始化 `bound = h`
      2. 循环调用 `ida_star_search`，若找到目标则返回解；否则更新 `bound` 为当前轮返回的最小超界限值
      3. 若无可扩展节点且未找到目标，最终返回无解

### **3部分关键代码展示**

展示启发式函数的计算代码以及$A*$算法和$IDA*$算法的搜索代码，以及$A*$算法中邻居的生成代码

#### **3.1曼哈顿距离计算**

In [2]:
def get_goal_position(goal_state):
    goal_positions = {}
    for i in range(4):
        for j in range(4):
            goal_positions[goal_state[i][j]] = (i, j)
    return goal_positions

def manhattan_distance(state, goal_positions):
    distance = 0
    for i in range(4):
        for j in range(4):
            if state[i][j] != 0:
                x_goal, y_goal = goal_positions[state[i][j]]
                distance += abs(i - x_goal) + abs(j - y_goal)
    return distance

#### **3.2线性冲突距离计算**

In [3]:
def linear_conflict(state, goal_positions):
    conflict = 0
    for i in range(4):
        for j in range(4):
            for k in range(j+1, 4):
                if state[i][j] != 0 and state[i][k] != 0:
                    if goal_positions[state[i][j]][0] == i and goal_positions[state[i][k]][0] == i:
                        if goal_positions[state[i][j]][1] > goal_positions[state[i][k]][1]:
                            conflict += 1
    for j in range(4):
        for i in range(4):
            for k in range(i+1, 4):
                if state[i][j] != 0 and state[k][j] != 0:
                    if goal_positions[state[i][j]][1] == j  and goal_positions[state[k][j]][1] == j:
                        if goal_positions[state[i][j]][0] > goal_positions[state[k][j]][0]:
                            conflict += 1
    return conflict

def heuristic(state, goal_positions):
    return manhattan_distance(state, goal_positions) + 2 * linear_conflict(state, goal_positions)

#### **3.3邻居节点扩展**

In [4]:
def get_neighbor(state):
    neighbors = []
    moves = {
        "Up" : (-1, 0),
        "Down": (1, 0),
        "Right": (0, 1),
        "Left": (0, -1)
    }
    x_blank, y_blank = -1, -1
    for i in range(4):
        for j in range(4):
            if state[i][j] == 0:
                x_blank = i
                y_blank = j
                break
    for move, (dx, dy) in moves.items():
        x_new = x_blank + dx
        y_new = y_blank + dy
        if 0 <= x_new < 4 and 0 <= y_new < 4:
            new_state = [list(row) for row in state]
            tmp = new_state[x_new][y_new]
            new_state[x_new][y_new] = new_state[x_blank][y_blank]
            new_state[x_blank][y_blank] = tmp
            neighbors.append((tuple(tuple(row) for row in new_state), move))
    return neighbors

#### **3.4 A*搜索**

In [5]:
def astar_puzzle(start_state, goal_state):
    open_set = []
    closed_set = set()
    goal_positions = get_goal_position(goal_state)
    start_node = PuzzleNode(
        start_state,
        parent=None,
        move=None,
        depth=0,
        cost=heuristic(start_state, goal_positions)
    )
    heapq.heappush(open_set, start_node)
    while open_set:
        current_node = heapq.heappop(open_set)

        if current_node.state == goal_state:
            path = []
            while current_node:
                if current_node.move:
                    path.append(current_node.move)
                current_node = current_node.parent
            return path[::-1]       
        closed_set.add(current_node.state)
        for neighbor_state, move in get_neighbor(current_node.state):
            if neighbor_state in closed_set:
                continue

            g_new = current_node.depth + 1
            h_new = heuristic(neighbor_state, goal_positions)
            new_node = PuzzleNode(
                neighbor_state,
                parent=current_node,
                move=move,
                depth=g_new,
                cost=g_new + h_new
            )
            heapq.heappush(open_set, new_node)
    return None

#### **3.5 IDA*搜索**

In [6]:
def ida_star_puzzle(start_state, goal_state):
    goal_positions = get_goal_position(goal_state)
    bound = manhattan_distance(start_state, goal_positions)

    path = {start_state}
    path_moves = []
    while True:
        t = ida_star_search(start_state, 0, bound, path, path_moves, goal_state, goal_positions)
        if isinstance(t, list):
            return t
        if t == float('inf'):
            return None
        bound = t 

def ida_star_search(state, g, bound, path, path_moves, goal_state, goal_positions):
    f = g + manhattan_distance(state, goal_positions)
    if f > bound:
        return f
    if state == goal_state:
        return path_moves.copy()
    min_bound = float('inf')
    for neighbor, move in get_neighbor(state):
        if neighbor in path:
            continue
        path.add(neighbor)
        path_moves.append(move)
        t = ida_star_search(neighbor, g + 1, bound, path, path_moves, goal_state, goal_positions)
        if isinstance(t, list):
            return t
        if t < min_bound:
            min_bound = t
        path.remove(neighbor)
        path_moves.pop()
    return min_bound

## **三、实验结果分析**

由于15-puzzle问题搜索解的空间极大，采用$A*$算法需要占用的内存空间极大，采用$IDA*$算法虽然可以减少内存的占用，但是可能重复搜索导致需要的搜索时间极大，故下面不完全展示所有的搜索结果。

### **1 A*算法结果**

$A*$算法中，采用`manhattan`距离作为启发函数可能内存开销极大导致无法搜索到正确结果，采用`liner_conflict`作为启发函数可以搜索到正确结果，但是用时较久。

#### **1.1 曼哈顿距离**

实现代码保存为文件`astar_manhattan.py`中，入口函数为`astar_puzzle`，测试文件为`test_astar.py`。

In [ ]:
from astar_manhattan import astar_puzzle
import time

# 使用time模块进行搜索时间统计
# 由于搜索内存开销与时间问题，不展示样例5，6

if __name__ == '__main__':
    goal_state = (
        (1, 2, 3, 4),
        (5, 6, 7, 8),
        (9, 10, 11, 12),
        (13, 14, 15, 0)
    )
    samples = [
        # test1
        (
            (1,  2,  4,  8),
            (5,  7, 11, 10),
            (13, 15, 0,  3),
            (14, 6,  9, 12)
        ),
        # test2
        (
            (14, 10, 6,  0),
            (4,   9,  1,  8),
            (2,   3,  5, 11),
            (12, 13,  7, 15)
        ),
        # test3
        (
            (5,  1,  3,  4),
            (2,  7,  8, 12),
            (9,  6, 11, 15),
            (13, 10, 14, 0)
        ),
        # test4
        (
            (6,  10,  3, 15),
            (14,  8,  7, 11),
            (5,   1,  0,  2),
            (13, 12,  9,  4)
        ),
    ]
    print("manhattan:")
    for i, puzzle in enumerate(samples, start=1):
        print(f"=== 测试样例 {i} ===")
        start_time = time.perf_counter()
        result = astar_puzzle(puzzle, goal_state)
        end_time = time.perf_counter()
        search_time = end_time - start_time
        if result is None:
            print("无解")
        else:
            print("解法步骤：", result)
        print("搜索用时：{:.4f} 秒".format(search_time))
        print()

#### **1.2线性冲突**
实现代码保存为文件`astar_linear_conflict.py`中，入口函数为`astar_puzzle`，测试文件为`test_astar.py`。

In [ ]:
from astar_linear_conflict import astar_puzzle
import time

# 使用time模块进行搜索时间统计
# 显示样例5，6但是搜索时间较长

if __name__ == '__main__':
    goal_state = (
        (1, 2, 3, 4),
        (5, 6, 7, 8),
        (9, 10, 11, 12),
        (13, 14, 15, 0)
    )
    samples = [
        (
            (1,  2,  4,  8),
            (5,  7, 11, 10),
            (13, 15, 0,  3),
            (14, 6,  9, 12)
        ),
        (
            (14, 10, 6,  0),
            (4,   9,  1,  8),
            (2,   3,  5, 11),
            (12, 13,  7, 15)
        ),
        (
            (5,  1,  3,  4),
            (2,  7,  8, 12),
            (9,  6, 11, 15),
            (13, 10, 14, 0)
        ),
        (
            (6,  10,  3, 15),
            (14,  8,  7, 11),
            (5,   1,  0,  2),
            (13, 12,  9,  4)
        ),
        (
            (11,  3,  1,  7),
            (4,   6,  8,  2),
            (15,  9, 10, 13),
            (14, 12,  5,  0)
        ),
        (
            (0,   5, 15, 14),
            (7,   9,  6, 13),
            (1,   2, 12, 10),
            (8,  11,  4,  3)
        ),
    ]
    print("manhattan:")
    for i, puzzle in enumerate(samples, start=1):
        print(f"=== 测试样例 {i} ===")
        start_time = time.perf_counter()
        result = astar_puzzle(puzzle, goal_state)
        end_time = time.perf_counter()
        search_time = end_time - start_time
        if result is None:
            print("无解")
        else:
            print("解法步骤：", result)
        print("搜索用时：{:.4f} 秒".format(search_time))
        print()

### **2 IDA*算法**

在$IDA*$算法中，`manhattan`距离的启发函数与`linear_conflict`的启发函数搜索时间都很长，但是由于$IDA*$本身搜索深度阈值的设置，其内存开销较小，可以通过大量运行时间得到搜索结果。

#### **2.1曼哈顿距离**
实现代码保存为文件`idasatr_manhattan.py`中，入口函数为`ida_star_puzzle`，测试文件为`test_idastar.py`。

In [ ]:
from idastar_manhattan import ida_star_puzzle
import time

# 使用time模块进行搜索时间统计
# 显示样例5，6但是搜索时间较长

if __name__ == '__main__':
    goal_state = (
        (1, 2, 3, 4),
        (5, 6, 7, 8),
        (9, 10, 11, 12),
        (13, 14, 15, 0)
    )
    samples = [
        (
            (1,  2,  4,  8),
            (5,  7, 11, 10),
            (13, 15, 0,  3),
            (14, 6,  9, 12)
        ),
        (
            (14, 10, 6,  0),
            (4,   9,  1,  8),
            (2,   3,  5, 11),
            (12, 13,  7, 15)
        ),
        (
            (5,  1,  3,  4),
            (2,  7,  8, 12),
            (9,  6, 11, 15),
            (13, 10, 14, 0)
        ),
        (
            (6,  10,  3, 15),
            (14,  8,  7, 11),
            (5,   1,  0,  2),
            (13, 12,  9,  4)
        ),
    ]
    print("manhattan:")
    for i, puzzle in enumerate(samples, start=1):
        print(f"=== 测试样例 {i} ===")
        start_time = time.perf_counter()
        result = ida_star_puzzle(puzzle, goal_state)
        end_time = time.perf_counter()
        search_time = end_time - start_time
        if result is None:
            print("无解")
        else:
            print("解法步骤：", result)
        print("搜索用时：{:.4f} 秒".format(search_time))
        print()

#### **2.2线性冲突**
实现代码保存为文件`idasatr_linear_conflict.py`中，入口函数为`ida_star_puzzle`，测试文件为`test_idastar.py`。

In [ ]:
from idasatr_linear_conflict import ida_star_puzzle
import time

# 使用time模块进行搜索时间统计
# 显示样例5，6但是搜索时间较长

if __name__ == '__main__':
    goal_state = (
        (1, 2, 3, 4),
        (5, 6, 7, 8),
        (9, 10, 11, 12),
        (13, 14, 15, 0)
    )
    samples = [
        (
            (1,  2,  4,  8),
            (5,  7, 11, 10),
            (13, 15, 0,  3),
            (14, 6,  9, 12)
        ),
        (
            (14, 10, 6,  0),
            (4,   9,  1,  8),
            (2,   3,  5, 11),
            (12, 13,  7, 15)
        ),
        (
            (5,  1,  3,  4),
            (2,  7,  8, 12),
            (9,  6, 11, 15),
            (13, 10, 14, 0)
        ),
        (
            (6,  10,  3, 15),
            (14,  8,  7, 11),
            (5,   1,  0,  2),
            (13, 12,  9,  4)
        ),
    ]
    print("manhattan:")
    for i, puzzle in enumerate(samples, start=1):
        print(f"=== 测试样例 {i} ===")
        start_time = time.perf_counter()
        result = ida_star_puzzle(puzzle, goal_state)
        end_time = time.perf_counter()
        search_time = end_time - start_time
        if result is None:
            print("无解")
        else:
            print("解法步骤：", result)
        print("搜索用时：{:.4f} 秒".format(search_time))
        print()

### **3性能分析**

由上面的输出结果可以看到，对于较为简单的问题，可以相对快速的得到最优解，但是一旦问题复杂化，需要搜索的时间快速增加。对于$A*$算法而言，需要的内存开销很大，对于$IDA*$算法而言，需要的搜索时间很大。故无法很好的正确求解15-puzzle问题

## **四、优化算法**

### **1优化内容**

由于采用一般的启发函数，曼哈顿距离，线性冲突，对于每个滑块的预测不准确，得到的启发值远低于实际情况。故可以考虑采用模式数据库(PDB)的启发函数。

### **2算法设计**

- **原理**：首先将15-puzzle问题分解为几个不同独立的子集，分别对每个子集进行分析。从目标结果进行反推，遍历所有的情况，并记录相应的步数，作为当前状态的h(n)。最后将分解的不同的子集中计算的每个状态的h(n)线性相加即得到最终需要的h(n)。由于每个子集的独立性，保证最终通过线性相加得到的h(n)一定不会高估与实际的步数。
- **实现**：由于pdb需要使用bfs遍历模拟所有的情况，故需要考虑内存与时间的问题，为了能尽可能高效的实现。现将15-puzzle问题分解为三组，理论上每个子集的状态数应该为：$$ C(16,6) \times 6!=5765760$$
而6-6-3的分组需要遍历的状态数为$57,679,440$，7-8分组为$5,189,184,000$。对比之下，5-5-5的分组相对合理。
故最终的$f(n)$计算为$$ f(n)=h_1(n)+h_2(n)+h_3(n)+g(n)$$
- **具体函数**：
  - `extract_pattern`：提取需要tile的部分，不需要的标记为-1
  - `build_pattern_db`：反向遍历构造pdb库，pdb为一个字典，键值为每一个`state`以及对应的`depth`
  - `pdb_heuristic`：线性叠加计算启发值，利用函数`get`取出对应子集状态下的值

### **3关键代码**

In [15]:
def extract_pattern(state, pattern_tiles):
    pattern_state = []
    for row in state:
        new_row = []
        for tile in row:
            if tile == 0 or tile in pattern_tiles:
                new_row.append(tile)
            else:
                new_row.append(-1)
        pattern_state.append(tuple(new_row))
    return tuple(pattern_state)

def build_pattern_db(pattern_tiles, goal_state):
    pdb = {}
    goal_pattern = extract_pattern(goal_state, set(pattern_tiles))
    pdb[goal_pattern] = 0
    queue = deque()
    queue.append(goal_state)
    while queue:
        current = queue.popleft()
        current_cost = pdb[extract_pattern(current, set(pattern_tiles))]
        for neighbor, move in get_neighbor(current):
            neighbor_pattern = extract_pattern(neighbor, set(pattern_tiles))
            if neighbor_pattern not in pdb:
                pdb[neighbor_pattern] = current_cost + 1
                queue.append(neighbor)
    return pdb

def pdb_heuristic(state, pdb1, pdb2, pdb3, tiles1, tiles2, tiles3):
    h1 = pdb1.get(extract_pattern(state, set(tiles1)), 0)
    h2 = pdb2.get(extract_pattern(state, set(tiles2)), 0)
    h3 = pdb3.get(extract_pattern(state, set(tiles3)), 0)
    return h1 + h2 + h3

### **4结果分析**

PDB方法的实现代码存放在`astar_pdb.py`中，测试代码为`test_pdb.py`，启动下面的测试代码，运行得到结果如下(构造PDB库用时为总用时)

In [ ]:
import time
from astar_pdb import load_or_generate_pdb, astar_puzzle

if __name__ == '__main__':
    goal_state = (
        (1, 2, 3, 4),
        (5, 6, 7, 8),
        (9, 10, 11, 12),
        (13, 14, 15, 0)
    )
    tiles1 = [1, 2, 3, 4, 5]
    tiles2 = [6, 7, 8, 9, 10]
    tiles3 = [11, 12, 13, 14, 15]
    pdb_start_time = time.perf_counter()
    print("正在构建或加载 PDB1 ...")
    pdb1 = load_or_generate_pdb(goal_state, tiles1, "pdb_1.pkl")
    print("PDB1 状态数：", len(pdb1))
    print("正在构建或加载 PDB2 ...")
    pdb2 = load_or_generate_pdb(goal_state, tiles2, "pdb_2.pkl")
    print("PDB2 状态数：", len(pdb2))
    print("正在构建或加载 PDB3 ...")
    pdb3 = load_or_generate_pdb(goal_state, tiles3, "pdb_3.pkl")
    print("PDB3 状态数：", len(pdb3))
    pdb_end_time = time.perf_counter()
    print("PDB生成/加载时间：{:.4f}秒".format(pdb_end_time - pdb_start_time))
    print()
    samples = [
        (
            (1,  2,  4,  8),
            (5,  7, 11, 10),
            (13, 15, 0,  3),
            (14, 6,  9, 12)
        ),
        (
            (14, 10, 6,  0),
            (4,   9,  1,  8),
            (2,   3,  5, 11),
            (12, 13,  7, 15)
        ),
        (
            (5,  1,  3,  4),
            (2,  7,  8, 12),
            (9,  6, 11, 15),
            (13, 10, 14, 0)
        ),
        (
            (6,  10,  3, 15),
            (14,  8,  7, 11),
            (5,   1,  0,  2),
            (13, 12,  9,  4)
        ),
        (
            (11,  3,  1,  7),
            (4,   6,  8,  2),
            (15,  9, 10, 13),
            (14, 12,  5,  0)
        ),
        (
            (0,   5, 15, 14),
            (7,   9,  6, 13),
            (1,   2, 12, 10),
            (8,  11,  4,  3)
        ),
    ]
    for i, puzzle in enumerate(samples, start=1):
        print(f"=== 测试样例 {i} ===")
        search_start_time = time.perf_counter()
        result = astar_puzzle(puzzle, goal_state, pdb1, pdb2, pdb3, tiles1, tiles2, tiles3)
        search_end_time = time.perf_counter()
        if result is None:
            print("无解")
        else:
            print("解法步骤：", result)
        print("搜索时间：{:.4f}秒".format(search_end_time - search_start_time))
        print()

### **5性能分析**

构造PDB的时间需要271秒，在构造完成后可以多次使用，只需加载即可。进入$A*$搜索部分，最终的搜索时间对上述的所有样例都非常短，说明PDB方法启发函数的有效性。

## **五、参考资料**

1. https://github.com/metalglove/15-Puzzle
2. https://github.com/mwong510ca/15Puzzle_OptimalSolver
3. https://github.com/MilanPecov/15-Puzzle-Solvers